<a href="https://colab.research.google.com/github/aravindakrishnanl/thirukkural/blob/main/Graph_rag_tkipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --quiet datasets pandas networkx sentence-transformers faiss-cpu transformers langdetect deep-translator

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 7.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 59.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 98.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 88.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.7 MB/s eta 0:00:00
   

In [ ]:
import os
import time
from datasets import load_dataset
import pandas as pd
import networkx as nx
from sentence_transformers import SentenceTransformer, util
import faiss
from langdetect import detect
from deep_translator import GoogleTranslator
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

def detect_lang(text):
    try:
        return detect(text)
    except:
        return "en"


In [ ]:
print("Loading dataset from HuggingFace...")
ds = load_dataset("Selvakumarduraipandian/Thirukural", split="train")
df = ds.to_pandas()

print(df.columns)
print("Rows:", len(df))

df['Kural'] = df['Kural'].astype(str).str.replace('<br />',' ').str.strip()

df['kural_id'] = df['Adhigaram_ID'].astype(str) + "." + df['ID'].astype(str)
df.head(3)


Loading dataset from HuggingFace...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/1.52M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1330 [00:00<?, ? examples/s]

Index(['ID', 'Adhigaram_ID', 'Paal', 'Iyal', 'Adhigaram', 'Kural',
       'Transliteration', 'Vilakam', 'Couplet', 'Chapter', 'Section',
       'Athigaram', 'Kalaingar_Urai', 'Parimezhalagar_Urai',
       'M_Varadharajanar', 'Solomon_Pappaiya'],
      dtype='object')
Rows: 1330


,ID,Adhigaram_ID,Paal,Iyal,Adhigaram,Kural,Transliteration,Vilakam,Couplet,Chapter,Section,Athigaram,Kalaingar_Urai,Parimezhalagar_Urai,M_Varadharajanar,Solomon_Pappaiya,kural_id
0,1,1,அறத்துப்பால்,பாயிரவியல்,கடவுள் வாழ்த்து,அகர முதல எழுத்தெல்லாம் ஆதி பகவன் முதற்றே உலகு.,Akara Mudhala Ezhuththellaam Aadhi Pakavan ...,"அகரம் எழுத்துக்களுக்கு முதன்மை; ஆதிபகவன், உலகி...","A, as its first of letters, every speech maint...",அறத்துப்பால்,1.1,கடவுள் வாழ்த்து,"அகரம் எழுத்துக்களுக்கு முதன்மை; ஆதிபகவன்,...",எழுத்து எல்லாம் அகரம் முதல - எழுத்துக்கள் எல்ல...,எழுத்துக்கள் எல்லாம் அகரத்தை அடிப்படையாக கொண்...,எழுத்துக்கள் எல்லாம் அகரத்தில் தொடங்குகின்றன; ...,1.1
1,2,1,அறத்துப்பால்,பாயிரவியல்,கடவுள் வாழ்த்து,கற்றதனால் ஆய பயனென்கொல் வாலறிவன் நற்றாள் தொழாஅ...,Katradhanaal Aaya Payanenkol Vaalarivan Nat...,தன்னைவிட அறிவில் மூத்த பெருந்தகையாளரின் முன்னே...,"No fruit have men of all their studied lore, ...",அறத்துப்பால்,1.2,கடவுள் வாழ்த்து,தன்னைவிட அறிவில் மூத்த பெருந்தகையாளரின் முன்ன...,கற்றதனால் ஆய பயன் என் - எல்லா நூல்களையும் கற்ற...,தூய அறிவு வடிவாக விளங்கும் இறைவனுடைய நல்ல திரு...,"தூய அறிவு வடிவானவனின் திருவடிகளை வணங்காதவர், ப...",1.2
2,3,1,அறத்துப்பால்,பாயிரவியல்,கடவுள் வாழ்த்து,மலர்மிசை ஏகினான் மாணடி சேர்ந்தார் நிலமிசை நீடு...,Malarmisai Ekinaan Maanati Serndhaar Nilami...,மலர் போன்ற மனத்தில் நிறைந்தவனைப் பின்பற்றுவோரி...,"His feet, 'Who o'er the full-blown flower hath...",அறத்துப்பால்,1.3,கடவுள் வாழ்த்து,மலர் போன்ற மனத்தில் நிறைந்தவனைப் பின்பற்றுவோரி...,மலர்மிசை ஏகினான் மாண்அடி சேர்ந்தார் - மலரின் க...,அன்பரின் அகமாகிய மலரில் வீற்றிருக்கும் கடவுளின...,மனமாகிய மலர்மீது சென்று இருப்பவனாகிய கடவுளின் ...,1.3


In [ ]:
print("Building knowledge graph...")
G = nx.Graph()

for _, row in df.iterrows():
    node_id = f"Kural {row['kural_id']}"

    G.add_node(node_id, type="kural", text=row['Kural'], chapter=row['Adhigaram'], section=row['Section'])

    chapter_node = f"Chapter: {row['Adhigaram']}"
    section_node = f"Section: {row['Section']}"
    G.add_node(chapter_node, type="chapter")
    G.add_node(section_node, type="section")

    G.add_edge(node_id, chapter_node)
    G.add_edge(chapter_node, section_node)

print("Graph built: nodes", G.number_of_nodes(), "edges", G.number_of_edges())



Building knowledge graph...
Graph built: nodes 2788 edges 2652


In [ ]:
print("Creating embeddings (paraphrase-multilingual-MiniLM-L12-v2)...")
embed_model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")
kural_texts = df["Kural"].tolist()
kural_ids = df["kural_id"].tolist()
embeddings = embed_model.encode(kural_texts, show_progress_bar=True, convert_to_numpy=True)


d = embeddings.shape[1]
index = faiss.IndexFlatIP(d)

import numpy as np
emb_norm = embeddings / np.linalg.norm(embeddings, axis=1, keepdims=True)
index.add(emb_norm)
print("FAISS index ready, items:", index.ntotal)


Creating embeddings (paraphrase-multilingual-MiniLM-L12-v2)...


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/42 [00:00<?, ?it/s]

FAISS index ready, items: 1330


In [ ]:
print("Adding similarity edges (top-3 neighbors per kural)...")
k = 3
hits = util.semantic_search(embeddings, embeddings, top_k=k)
for i, lst in enumerate(hits):
    src_node = f"Kural {kural_ids[i]}"
    for hit in lst[1:]:
        j = hit['corpus_id']
        sim_node = f"Kural {kural_ids[j]}"

        G.add_edge(src_node, sim_node, relation="SIMILAR_TO", score=float(hit['score']))
print("Similarity edges added.")


Adding similarity edges (top-3 neighbors per kural)...
Similarity edges added.


In [ ]:
print("Loading summarization/generation model (flan-t5-small)...")
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")
gen_model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small")

summarizer = pipeline("text2text-generation", model=gen_model, tokenizer=tokenizer, device=-1)


Loading summarization/generation model (flan-t5-small)...


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Device set to use cpu


In [ ]:
def ensure_tamil_query(query):
    lang = detect_lang(query)
    if lang == 'ta':
        return query, False
    else:

        try:
            tamil_q = GoogleTranslator(source='auto', target='ta').translate(query)
            return tamil_q, True
        except Exception as e:
            print("Translation failed, using original query:", e)
            return query, False


In [ ]:
def hybrid_query(query, top_k=3, hop=1, summarize=True):
    """
    Returns structured Tamil output:
      - 'query_tamil' : the Tamil query used
      - 'retrieved_kurals' : list of (kural_id, text)
      - 'expanded_kurals' : list after graph expansion
      - 'summary' : optional short Tamil summary generated by model
    """

    query_ta, translated = ensure_tamil_query(query)

    q_emb = embed_model.encode([query_ta], convert_to_numpy=True)
    q_emb = q_emb / np.linalg.norm(q_emb, axis=1, keepdims=True)
    D, I = index.search(q_emb, top_k)
    top_idxs = I[0].tolist()

    top_nodes = [f"Kural {kural_ids[i]}" for i in top_idxs]
    retrieved = [(kural_ids[i], kural_texts[i]) for i in top_idxs]


    related_nodes = set()
    for node in top_nodes:
        if node in G:

            related_nodes.add(node)

            neighbors = nx.single_source_shortest_path_length(G, node, cutoff=hop).keys()
            related_nodes.update(neighbors)

    expanded = []
    for n in related_nodes:
        if G.nodes[n].get('type') == 'kural' or n.startswith('Kural '):

            k_id = n.split("Kural ")[-1] if n.startswith("Kural ") else None
            if k_id:

                try:
                    idx = kural_ids.index(k_id)
                    expanded.append((k_id, kural_texts[idx], G.nodes[n].get('chapter'), G.nodes[n].get('section')))
                except ValueError:
                    pass


    result = {
        "original_query": query,
        "query_tamil": query_ta,
        "translated_from_english": translated,
        "retrieved": retrieved,
        "expanded": expanded
    }


    if summarize and len(expanded) > 0:

        context_lines = []
        for k_id, text, chapter, section in expanded[:10]:
            context_lines.append(f"[குறள் {k_id}] {text} ({chapter} - {section})")
        prompt = "கீழ் கொடுக்கப்பட்ட குறள்களின் பொருளைப் பொறுத்து சுருக்கமாக, தெளிவாக தமிழில் பதில் சொல்லவும்:\n\n"
        prompt += "\n".join(context_lines) + "\n\nபொருள்:"

        gen = summarizer(prompt, max_new_tokens=128, do_sample=False)
        summary = gen[0]['generated_text']
        result['summary'] = summary
    else:
        result['summary'] = None

    return result


In [ ]:
print("Example: Tamil query")
r1 = hybrid_query("திருக்குறள் நட்பு பற்றி என்ன கூறுகிறது", top_k=3, hop=1, summarize=True)
print("Query used (Tamil):", r1['query_tamil'])
print("Summary (Tamil):\n", r1['summary'])
print("\nExpanded kurals (sample):")
for k in r1['expanded'][:6]:
    print(k[0], ":", k[1])

print("\n\nExample: English query (auto-translate -> Tamil)")
r2 = hybrid_query("What does Thirukkural say about education", top_k=3, hop=1, summarize=True)
print("Query used (Tamil):", r2['query_tamil'])
print("Summary (Tamil):\n", r2['summary'])
print("\nExpanded kurals (sample):")
for k in r2['expanded'][:6]:
    print(k[0], ":", k[1])


Example: Tamil query
Query used (Tamil): திருக்குறள் நட்பு பற்றி என்ன கூறுகிறது
Summary (Tamil):
 [ 10.96]                                                              

Expanded kurals (sample):
10.96 : அல்லவை தேய அறம்பெருகும் நல்லவை நாடி இனிய சொலின்
84.833 : நாணாமை நாடாமை நாரின்மை யாதொன்றும் பேணாமை பேதை தொழில்
80.798 : உள்ளற்க உள்ளம் சிறுகுவ கொள்ளற்க அல்லற்கண் ஆற்றறுப்பார் நட்பு.
113.1122 : உடம்பொடு உயிரிடை என்னமற் றன்ன மடந்தையொடு எம்மிடை நட்பு.
79.786 : முகநக நட்பது நட்பன்று நெஞ்சத்து அகநக நட்பது நட்பு.
34.335 : நாச்செற்று விக்குள்மேல் வாராமுன் நல்வினை மேற்சென்று செய்யப் படும்


Example: English query (auto-translate -> Tamil)
Query used (Tamil): திருகுரல் கல்வியைப் பற்றி என்ன சொல்கிறது
Summary (Tamil):
 [ 97.963]        . ( - 97.3) [ 37.362]        . ( - 37.2) [ 106.1060]        . ( - 46.4) [ 111.1109]        . ( - 46.4) [ 11

Expanded kurals (sample):
97.963 : பெருக்கத்து வேண்டும் பணிதல் சிறிய சுருக்கத்து வேண்டும் உயர்வு.
37.362 : வேண்டுங்கால் வேண்டும் பிறவாமை மற்றது வேண்டாமை வேண்